In [5]:
import ast
import numpy as np
import pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

In [9]:
train_df = pd.read_csv('/datadisk/kg/siim/tmp/train_df.csv')

train_df["boxes"].fillna("[{'x':0,'y':0,'width':1,'height':1}]",inplace=True)
train_df["boxes"] = train_df["boxes"].map(lambda x : ast.literal_eval(x))

def preprocess_bbox(row):
    factor_x = 1/row[9]
    factor_y = 1/row[8]
    bboxes = []
    
    if row[2].split(' ')[0] == "opacity":
        for box in row[1]:
            x = box["x"] * factor_x
            y = box["y"] * factor_y
            w = box["width"] * factor_x
            h = box["height"] * factor_y
            xc = x + w/2
            yc = y + h/2
        
            bboxes.append([xc,yc,w,h])
    return bboxes

LABEL_PATH = "labels"
os.makedirs(LABEL_PATH, exist_ok=True)
for row in train_df.values:
    filename = row[0].split('_')[0]
    filepath = f"{LABEL_PATH}/{filename}.txt"
    
    label = row[2].split(' ')[0]
    if label == "opacity":
        bbox = preprocess_bbox(row)
        with open(filepath, "w") as f:
            for box in bbox:
                box = [0] + box
                box = [str(i) for i in box]
                box = ' '.join(box)
                f.write(box)
                f.write('\n')
    else:
        with open(filepath, "w") as f:
            pass